In [2]:
import os
import numpy as np
import tensorflow as tf
import cv2
import pickle
# import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

In [8]:
images = []
labels = [] # dogs = 1 # wolfs = 0

for root, dirs, files in os.walk('data'):
    for name in files:
        if name.split('.')[1] == 'jpeg':
            file_name = os.path.join(root, name)
            labels.append(int('dogs' in file_name))
            image = cv2.imread(file_name, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)

with open('dog_wolf_files.sav', 'wb') as outfile:
    pickle.dump((labels, images), outfile)

In [9]:
with open('dog_wolf_files.sav', 'rb') as infile:
    labels, images = pickle.load(infile)

In [10]:
class ImageScaler:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def scale(self, images):
        if type(images) is list:
            image_list = [self.scale(image) for image in images]
            return tf.reshape(np.array(image_list), [-1, self.width, self.height, 3, 1]) # TODO: manage channel number prettier
        else:
            return cv2.resize(images, (self.height, self.width)) # TODO: resize from [0..255] to [0..1]

In [11]:
scaler = ImageScaler(width=50, height=50)
scaled_images = scaler.scale(images)

In [12]:
np.array(scaled_images).shape

(24, 50, 50, 3, 1)

In [13]:
train_images, test_images, train_labels, test_labels = train_test_split(scaled_images, labels, shuffle=True)
# TODO: test different ratios

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 4, 20, 11, 21,  1, 18, 10, 12,  5, 19, 23,  6, 16, 15, 17,  9, 22,
       14])

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(scaler.height, scaler.width, 3)), #TODO: Change 3 to 1 if GrayScale
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.fit(scaled_images, np.array(train_labels), epochs=10)

In [ ]:
model.predict()